# Find similar objects in XMM for objects in simbad, GEV and TEV catalogs
First, we import all the packages that we will need.

In [1]:
import warnings
import pandas as pd
import numpy as np
from getsimbad import get_simbad_data
from gevtev import compare_gev_tev_data, compare_gev_tev_data2
from astropy.io import fits
from definitions import *

warnings.filterwarnings("ignore")

This constant determines a threshold for the similarity between the objects:
$$|glat_{1} - glat_{2}| <= \epsilon$$
$$\epsilon = 0,5*10^{-1}$$

In [2]:
_epsilon = 0.12
_epsilon2 = 0.05

The cell above loads data from SIMBAD catalog obtained by requests 'Be', 'O', 'B' and 'plsr'. If the file "data/symbad.txt" does not exist, run getsimbad.py.

In [4]:
try:
    pd_simbad = pd.read_csv("data/simbad.txt", sep='\t', encoding='utf-8')
    del pd_simbad['Unnamed: 0']
except FileNotFoundError:
    pd_simbad = get_simbad_data()


In [5]:
pd_simbad.head()

,s_MAIN_ID,s_RA,s_DEC,s_RA_PREC,s_DEC_PREC,s_COO_ERR_MAJA,s_COO_ERR_MINA,s_COO_ERR_ANGLE,s_COO_QUAL,s_COO_WAVELENGTH,...,s_FLUX_u,s_FLUX_g,s_FLUX_r,s_FLUX_i,s_FLUX_z,s_class,s_glat,s_glon,s_nan_spectra,s_zero_spectra
0,b'HD 201433A',21 08 38.894,+30 12 20.27,7.0,7.0,26.0000,26.0000,90.0,B,O,...,NaN,NaN,NaN,NaN,NaN,plsr,-11.774311,76.107743,False,False
1,b'* nu. For',02 04 29.4462,-29 17 48.491,14.0,14.0,0.1834,0.1942,90.0,A,O,...,NaN,NaN,NaN,NaN,NaN,plsr,-73.777993,224.887601,False,False
2,b'* phi Her',16 08 46.1668,+44 56 05.732,14.0,14.0,0.3710,0.3754,90.0,A,O,...,NaN,NaN,NaN,NaN,NaN,plsr,47.107400,70.847825,False,False
3,b'V* CG And',00 00 43.6345,+45 15 12.002,14.0,14.0,0.0550,0.0422,90.0,A,O,...,NaN,NaN,NaN,NaN,NaN,plsr,-16.702455,113.649358,False,False
4,b'HD 133029',15 00 38.7184,+47 16 38.793,14.0,14.0,0.0472,0.0421,90.0,A,O,...,NaN,NaN,NaN,NaN,NaN,plsr,57.710074,80.191942,False,False


In [3]:
s_spectrum_columns = [
    's_FLUX_U', 
    's_FLUX_B', 
    's_FLUX_V', 
    's_FLUX_R', 
    's_FLUX_I',
    's_FLUX_G', 
    's_FLUX_J', 
    's_FLUX_H',
    's_FLUX_K', 
    's_FLUX_u',
    's_FLUX_g', 
    's_FLUX_r',
    's_FLUX_i',    
    's_FLUX_z',
    ]

In [4]:
_path_xmm = 'data/3XMM_DR7cat_slim_v1.0.fits.gz'
_names_xmm = [
    "SC_RA",
    "SC_DEC",
    "SC_POSERR",
    "SC_EP_1_FLUX",
    "SC_EP_2_FLUX",
    "SC_EP_3_FLUX",
    "SC_EP_4_FLUX",
    "SC_EP_5_FLUX",
    "SC_EP_1_FLUX_ERR",
    "SC_EP_2_FLUX_ERR",
    "SC_EP_3_FLUX_ERR",
    "SC_EP_4_FLUX_ERR",
    "SC_EP_5_FLUX_ERR",
    ] 

In [5]:
hdul_xmm = fits.open(_path_xmm)
cat_xmm = hdul_xmm[1].data

In [6]:
def create_pandas_frames(cat, cat_name):   
    """
    Creates pandas dataframes with the same values as in cat, adding to columns cat_name.
    
    Arguments:
    cat -- rec array with catalog data
    cat_name -- short catalog name
    
    Returns:
    data -- pandas DataFrame with catalog data
    """
    
    data = pd.DataFrame.from_records(cat.tolist(), columns=cat.dtype.names)
    match_names = {}
    for i in data.columns:
        match_names.update({i : cat_name + i})
    data = data.rename(columns = match_names)
    
    return data

In [7]:
data_xmm = create_pandas_frames(cat_xmm, "xmm_")
data_xmm.head()

,xmm_SRCID,xmm_IAUNAME,xmm_SC_RA,xmm_SC_DEC,xmm_SC_POSERR,xmm_SC_DET_ML,xmm_SC_EP_1_FLUX,xmm_SC_EP_1_FLUX_ERR,xmm_SC_EP_2_FLUX,xmm_SC_EP_2_FLUX_ERR,...,xmm_SC_SUM_FLAG,xmm_SC_EP_8_FMIN,xmm_SC_EP_8_FMIN_ERR,xmm_SC_EP_8_FMAX,xmm_SC_EP_8_FMAX_ERR,xmm_MJD_FIRST,xmm_MJD_LAST,xmm_N_DETECTIONS,xmm_CONFUSED,xmm_WEBPAGE_URL
0,206931901010113,b'3XMM J000000.0-552108',0.000395,-55.352442,2.31261,11.961400,5.975190e-17,1.080930e-16,7.739470e-16,2.520810e-16,...,0,6.018350e-15,5.191330e-15,6.018350e-15,5.191330e-15,56086.742951,56087.588194,1,70,b'http://xmm-catalog.irap.omp.eu/source/206931...
1,204033901010030,b'3XMM J000000.2+623122',0.001138,62.523022,1.81998,24.577000,2.084380e-17,6.754120e-17,1.423010e-17,9.893530e-17,...,0,7.443400e-15,1.952370e-15,7.443400e-15,1.952370e-15,54134.485278,54135.043600,1,70,b'http://xmm-catalog.irap.omp.eu/source/204033...
2,201253101010082,b'3XMM J000000.3-250629',0.001307,-25.108284,2.15064,10.571600,1.946440e-16,2.242400e-16,1.116210e-15,3.949710e-16,...,0,5.887980e-15,3.771330e-15,5.887980e-15,3.771330e-15,51696.440880,51696.788090,1,70,b'http://xmm-catalog.irap.omp.eu/source/201253...
3,206931901010049,b'3XMM J000000.3-552139',0.001344,-55.361047,1.86001,48.571301,7.582340e-16,2.276970e-16,1.875440e-15,3.475400e-16,...,0,1.669370e-14,6.452240e-15,1.669370e-14,6.452240e-15,56086.742951,56087.588194,1,70,b'http://xmm-catalog.irap.omp.eu/source/206931...
4,203025805010073,b'3XMM J000000.7-321352',0.003264,-32.231256,1.94055,14.014100,2.411770e-16,1.397010e-16,7.000080e-16,2.408750e-16,...,0,6.316080e-15,3.759480e-15,6.316080e-15,3.759480e-15,53534.288912,53534.827627,1,70,b'http://xmm-catalog.irap.omp.eu/source/203025...


In [ ]:
del cat_xmm

Load data from gevtevsimbad, gevsimbad and tevsimbad:

In [11]:
# compare using glat glon coords
common_data, only_tev_data, only_gev_data = compare_gev_tev_data(_epsilon)
print(common_data.shape, only_tev_data.shape, only_gev_data.shape)

(111, 166) (166, 89) (3034, 79)


In [10]:
# compare using ra dec coords
common_data, only_tev_data, only_gev_data = compare_gev_tev_data2(_epsilon)
print(common_data.shape, only_tev_data.shape, only_gev_data.shape)

(110, 166) (166, 89) (3034, 79)


In [15]:
def angsep(ra1, dec1, ra2, dec2):
    SEP = np.cos(dec1 * np.pi / 180) * np.cos(dec2 * np.pi / 180) * np.cos((ra1 - ra2) * np.pi / 180)
    SEP += np.sin(dec1 * np.pi / 180) * np.sin(dec2 * np.pi / 180) #returns values between 0 and pi radians
    SEP = np.arccos(SEP)
    return SEP * 180. / np.pi

In [12]:
def create_matrix_common(cat1, cat2, epsilon, cat1_col1, cat1_col2, cat2_col1, cat2_col2):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a matrix with association: m[i][j] = True if object i from cat1
            is close enough to object j from cat2
    
    cat1, cat2 - a pandas dataFrames 
    epsilon(double) - distance accepted as equivalence
    """
    
    col1_dif_matrix = np.dot(np.vstack((cat1[cat1_col1], -np.ones_like(cat1[cat1_col1]))).T,
                            np.vstack((np.ones_like(cat2[cat2_col1]), cat2[cat2_col1])))
    col2_dif_matrix = np.dot(np.vstack((cat1[cat1_col2], -np.ones_like(cat1[cat1_col2]))).T,
                            np.vstack((np.ones_like(cat2[cat2_col2]), cat2[cat2_col2])))
    pairs_matrix = np.logical_and(np.abs(col1_dif_matrix) < epsilon, 
                                np.abs(col2_dif_matrix) < epsilon)

    return pairs_matrix

In [13]:
def create_matrix_common2(data1, data2, epsilon, data1_ra, data1_dec, data2_ra, data2_dec):
    """This function looks for the same objects in GeV and TeV and Simbad catalogs 
    
    Return: a matrix with association: m[i][j] = True if object i from cat1
            is close enough to object j from cat2
    
    cat1, cat2 - a pandas dataFrames 
    epsilon(double) - distance accepted as equivalence
    """

    ra_dif_matrix = np.dot(np.vstack((data1[data1_ra].T, -np.ones_like(data1[data1_ra].T))).T, 
                           np.vstack((np.ones_like(data2[data2_ra].T), data2[data2_ra].T)))
    print(ra_dif_matrix.shape)
    dec1 = data1[data1_dec].reshape(-1, 1)
    dec2 = data2[data2_dec].reshape(-1, 1)
    print(dec1.shape, dec2.shape)
#     print((np.cos(dec1 * np.pi / 180) * np.cos(dec2.T * np.pi / 180)).shape)

    SEP = np.cos((ra_dif_matrix) * np.pi / 180) * np.cos(dec1 * np.pi / 180) * np.cos(dec2.T * np.pi / 180)
    SEP += np.sin(dec1 * np.pi / 180) * np.sin(dec2.T * np.pi / 180) # returns values between 0 and pi radians
    SEP = np.arccos(SEP) * 180. / np.pi
     
    pairs_matrix = (SEP < epsilon)
    return pairs_matrix

In [14]:
from tqdm import tqdm

In [15]:
def create_common_data(data1, data2, pairs_matrix, cat1_col1, cat1_col2, cat2_col1, cat2_col2):
    """The function adds objects found in both catalogs.
    """
    vector_association = np.where(np.sum(pairs_matrix, axis=0) > 0)[0]
    pd_common = pd.DataFrame()
    for i in tqdm(vector_association):
        data1_join = (pairs_matrix[:, i] > 0) * (i + 1) - 1
        data1["join"] = data1_join
        pd_common0 = pd.merge(data1, data2, right_index=True, left_on='join', how='inner')
        if (len(pd_common)):
            pd_common = pd_common.append(pd_common0)
            del pd_common["join"]
            del data1["join"]
        else:
            pd_common = pd_common0.copy()
    
      
    array_non_duplicate = [cat1_col1, cat1_col2, cat2_col1, cat2_col2]
    pd_common = pd_common.drop_duplicates(array_non_duplicate)
#     pd_common = pd_common.reset_index()

    return pd_common

In [15]:
common_data.shape, only_tev_data.shape, only_gev_data.shape

((110, 166), (166, 89), (3034, 79))

In [23]:
matrix = create_matrix_common(common_data, pd_simbad, _epsilon, 'tev_glat', 'tev_glon', "s_glat", "s_glon")
pd_common_gevtevsimbad = create_common_data(common_data, pd_simbad, matrix, 'tev_glat', 'tev_glon', "s_glat", "s_glon")
print("There are " + str(len(pd_common_gevtevsimbad)) + " common objects in the three catalogs.")

100%|██████████| 67/67 [00:02<00:00, 30.77it/s]

There are 66 common objects in the three catalogs.


In [24]:
matrix = create_matrix_common2(pd_common_gevtevsimbad, data_xmm, _epsilon2, "gev_RAJ2000", "gev_DEJ2000", "xmm_SC_RA", "xmm_SC_DEC")

In [25]:
pd_common_gevtevsimbadxmm = create_common_data(pd_common_gevtevsimbad, data_xmm, matrix, "gev_RAJ2000", "gev_DEJ2000", "s_glat", "s_glon")
print("There are " + str(len(pd_common_gevtevsimbadxmm)) + " common objects in the three catalogs with xmm.")

100%|██████████| 200/200 [00:39<00:00,  5.03it/s]


There are 65 common objects in the three catalogs with xmm.


In [26]:
pd_common_gevtevsimbadxmm.head()

,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,xmm_SC_HR3,xmm_SC_HR3_ERR,xmm_SC_HR4,xmm_SC_HR4_ERR,xmm_SC_POSERR,xmm_SC_RA,xmm_SC_SUM_FLAG,xmm_SC_VAR_FLAG,xmm_SRCID,xmm_WEBPAGE_URL
12,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',b'LS I +61 303 ',...,0.504243,0.295949,-1.000000,0.107096,2.484910,40.106016,1,0,205059809010009,b'http://xmm-catalog.irap.omp.eu/source/205059...
36,b' ',b'1FGL J0956.5+6938 ',b' ',b'2FGL J0955.9+6936 ',b'M 82 ',b' ',b' ',b' ',b' ',b'M 82 ',...,-0.270341,0.119781,-0.537587,0.141592,0.824937,148.793632,3,0,201122902010063,b'http://xmm-catalog.irap.omp.eu/source/201122...
36,b' ',b'1FGL J0956.5+6938 ',b' ',b'2FGL J0955.9+6936 ',b'M 82 ',b' ',b' ',b' ',b' ',b'M 82 ',...,-0.270341,0.119781,-0.537587,0.141592,0.824937,148.793632,3,0,201122902010063,b'http://xmm-catalog.irap.omp.eu/source/201122...
36,b' ',b'1FGL J0956.5+6938 ',b' ',b'2FGL J0955.9+6936 ',b'M 82 ',b' ',b' ',b' ',b' ',b'M 82 ',...,-0.270341,0.119781,-0.537587,0.141592,0.824937,148.793632,3,0,201122902010063,b'http://xmm-catalog.irap.omp.eu/source/201122...
36,b' ',b'1FGL J0956.5+6938 ',b' ',b'2FGL J0955.9+6936 ',b'M 82 ',b' ',b' ',b' ',b' ',b'M 82 ',...,-0.270341,0.119781,-0.537587,0.141592,0.824937,148.793632,3,0,201122902010063,b'http://xmm-catalog.irap.omp.eu/source/201122...


The next cells find common values in simbad-tev

In [27]:
matrix = create_matrix_common(only_tev_data, pd_simbad, _epsilon, 'tev_glat', 'tev_glon', "s_glat", "s_glon")
pd_common_onlytevsimbad = create_common_data(only_tev_data, pd_simbad, matrix, 'tev_glat', 'tev_glon', "s_glat", "s_glon")
print("There are " + str(len(pd_common_onlytevsimbad)) + " common objects in the tev-simbad catalogs.")

100%|██████████| 126/126 [00:02<00:00, 42.81it/s]

There are 123 common objects in the tev-simbad catalogs.


In [28]:
matrix = create_matrix_common2(pd_common_onlytevsimbad, data_xmm, _epsilon2, "tev_ra", "tev_dec", "xmm_SC_RA", "xmm_SC_DEC")
matrix.shape

(123, 499266)

In [29]:
pd_common_onlytevsimbadxmm = create_common_data(pd_common_onlytevsimbad, data_xmm, matrix, "tev_ra", "tev_dec", "s_glat", "s_glon")
print("There are " + str(len(pd_common_onlytevsimbadxmm)) + " common objects in the tev-simbad catalog with xmm.")

100%|██████████| 454/454 [01:28<00:00,  5.12it/s]


There are 123 common objects in the tev-simbad catalog with xmm.


The next cells find common values in simbad-gev

In [16]:
matrix = create_matrix_common(only_gev_data, pd_simbad, _epsilon, 'gev_GLAT', 'gev_GLON', "s_glat", "s_glon")
pd_common_onlygevsimbad = create_common_data(only_gev_data, pd_simbad, matrix, 'gev_GLAT', 'gev_GLON', "s_glat", "s_glon")
print("There are " + str(len(pd_common_onlygevsimbad)) + " common objects in the gev-simbad catalogs.")

100%|██████████| 159/159 [00:03<00:00, 45.60it/s]

There are 155 common objects in the gev-simbad catalogs.


In [19]:
matrix = create_matrix_common2(pd_common_onlygevsimbad, data_xmm, _epsilon2, "gev_RAJ2000", "gev_DEJ2000", "xmm_SC_RA", "xmm_SC_DEC")
matrix.shape

(155, 499266)

In [20]:
pd_common_onlygevsimbadxmm = create_common_data(pd_common_onlygevsimbad, data_xmm, matrix, "gev_RAJ2000", "gev_DEJ2000", "s_glat", "s_glon")
print("There are " + str(len(pd_common_onlygevsimbadxmm)) + " common objects in the gev-simbad catalog with xmm.")

100%|██████████| 664/664 [02:06<00:00,  5.25it/s]

There are 105 common objects in the gev-simbad catalog with xmm.


In [21]:
pd_common_onlygevsimbadxmm.head()

,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,xmm_SC_HR3,xmm_SC_HR3_ERR,xmm_SC_HR4,xmm_SC_HR4_ERR,xmm_SC_POSERR,xmm_SC_RA,xmm_SC_SUM_FLAG,xmm_SC_VAR_FLAG,xmm_SRCID,xmm_WEBPAGE_URL
86,b' ',b' ',b'1FHL J0040.3+4049 ',b' ',b'B3 0037+405 ',b' ',b' ',b' ',b' ',b' ',...,-0.520912,0.330696,0.418585,0.517899,16.658001,10.018696,0,0,205057605010075,b'http://xmm-catalog.irap.omp.eu/source/205057...
86,b' ',b' ',b'1FHL J0040.3+4049 ',b' ',b'B3 0037+405 ',b' ',b' ',b' ',b' ',b' ',...,-0.520912,0.330696,0.418585,0.517899,16.658001,10.018696,0,0,205057605010075,b'http://xmm-catalog.irap.omp.eu/source/205057...
86,b' ',b' ',b'1FHL J0040.3+4049 ',b' ',b'B3 0037+405 ',b' ',b' ',b' ',b' ',b' ',...,-0.520912,0.330696,0.418585,0.517899,16.658001,10.018696,0,0,205057605010075,b'http://xmm-catalog.irap.omp.eu/source/205057...
86,b' ',b' ',b'1FHL J0040.3+4049 ',b' ',b'B3 0037+405 ',b' ',b' ',b' ',b' ',b' ',...,-0.520912,0.330696,0.418585,0.517899,16.658001,10.018696,0,0,205057605010075,b'http://xmm-catalog.irap.omp.eu/source/205057...
90,b' ',b' ',b' ',b'2FGL J0042.5+4114 ',b'M31 ',b' ',b' ',b' ',b' ',b' ',...,-0.338854,0.018973,-0.561577,0.028887,0.146908,10.563545,1,84,201125704010085,b'http://xmm-catalog.irap.omp.eu/source/201125...


In [30]:
pd_common_gevtevsimbadxmm.to_csv("data/gevtevsimbadxmm.txt", sep='\t')
pd_common_onlygevsimbadxmm.to_csv("data/gevsimbadxmm.txt", sep='\t')
pd_common_onlytevsimbadxmm.to_csv("data/tevsimbadxmm.txt", sep='\t')

In [22]:
# спектры из симбада
gevtevsimbad = pd.read_csv("data/gevtevsimbad.txt", sep='\t', encoding='utf-8', index_col=0)
for col in gevtevsimbad.columns:
    print(col)
gevtevsimbad.head()

level_0
gev_0FGL_Name
gev_1FGL_Name
gev_1FHL_Name
gev_2FGL_Name
gev_ASSOC1
gev_ASSOC2
gev_ASSOC_GAM1
gev_ASSOC_GAM2
gev_ASSOC_GAM3
gev_ASSOC_TEV
gev_CLASS1
gev_Conf_68_PosAng
gev_Conf_68_SemiMajor
gev_Conf_68_SemiMinor
gev_Conf_95_PosAng
gev_Conf_95_SemiMajor
gev_Conf_95_SemiMinor
gev_Cutoff
gev_DEJ2000
gev_Energy_Flux100
gev_Exp_Index
gev_Extended_Source_Name
gev_Flags
gev_Flux1000
gev_Flux10000_100000
gev_Flux1000_3000
gev_Flux100_300
gev_Flux3000_10000
gev_Flux300_1000
gev_Flux30_100
gev_Flux_Density
gev_Flux_History
gev_Flux_Peak
gev_GLAT
gev_GLON
gev_Peak_Interval
gev_Pivot_Energy
gev_PowerLaw_Index
gev_RAJ2000
gev_ROI_num
gev_Signif_Avg
gev_Signif_Curve
gev_Signif_Peak
gev_Source_Name
gev_Spectral_Index
gev_SpectrumType
gev_Sqrt_TS10000_100000
gev_Sqrt_TS1000_3000
gev_Sqrt_TS100_300
gev_Sqrt_TS3000_10000
gev_Sqrt_TS300_1000
gev_Sqrt_TS30_100
gev_TEVCAT_FLAG
gev_Time_Peak
gev_Unc_Cutoff
gev_Unc_Energy_Flux100
gev_Unc_Exp_Index
gev_Unc_Flux1000
gev_Unc_Flux10000_100000
gev_Unc_Flux

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_pl_norm_err,tev_spec_pl_norm_err_sys,tev_spec_theta,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where
0,91,b' ',b'1FGL J1614.7-5138c',b'1FHL J1615.3-5146e',b'2FGL J1615.2-5138 ',b' ',b' ',b' ',b' ',b' ',...,3.400000e-13,1.760000e-12,0.49,b'pl',b'ChXqoJ',234,b'TeV J1614-518',83,b'TeV J1614-5149',b'gal'
1,117,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'
2,178,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,1.200000e-13,1.500000e-13,0.15,b'pl',b'F0Luol',193,b'TeV J1930+188',136,b'TeV J1930+1852',b'gal'
3,160,b' ',b' ',b' ',b' ',b'LAT PSR J1838-0537 ',b' ',b' ',b' ',b' ',...,1.300000e-12,2.000000e-13,0.70,b'pl',b'amNIwH',142,b'TeV J1840-055',125,b'TeV J1840-0533',b'gal'
4,117,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,0.10,b'ecpl',b'UvKbRe',81,b'TeV J1745-290',106,b'TeV J1745-2900',b'gal'


In [23]:
gevtevsimbad.shape

(72, 197)

In [9]:
# без спектров из симбада
gevtev_simbadclasses = pd.read_csv("data/gevtev_simbadclasses.txt", sep='\t', encoding='utf-8', index_col=0)
for col in gevtev_simbadclasses.columns:
    print(col)
gevtev_simbadclasses.head()

index
gev_0FGL_Name
gev_1FGL_Name
gev_1FHL_Name
gev_2FGL_Name
gev_ASSOC1
gev_ASSOC2
gev_ASSOC_GAM1
gev_ASSOC_GAM2
gev_ASSOC_GAM3
gev_ASSOC_TEV
gev_CLASS1
gev_Conf_68_PosAng
gev_Conf_68_SemiMajor
gev_Conf_68_SemiMinor
gev_Conf_95_PosAng
gev_Conf_95_SemiMajor
gev_Conf_95_SemiMinor
gev_Cutoff
gev_DEJ2000
gev_Energy_Flux100
gev_Exp_Index
gev_Extended_Source_Name
gev_Flags
gev_Flux1000
gev_Flux10000_100000
gev_Flux1000_3000
gev_Flux100_300
gev_Flux3000_10000
gev_Flux300_1000
gev_Flux30_100
gev_Flux_Density
gev_Flux_History
gev_Flux_Peak
gev_GLAT
gev_GLON
gev_Peak_Interval
gev_Pivot_Energy
gev_PowerLaw_Index
gev_RAJ2000
gev_ROI_num
gev_Signif_Avg
gev_Signif_Curve
gev_Signif_Peak
gev_Source_Name
gev_Spectral_Index
gev_SpectrumType
gev_Sqrt_TS10000_100000
gev_Sqrt_TS1000_3000
gev_Sqrt_TS100_300
gev_Sqrt_TS3000_10000
gev_Sqrt_TS300_1000
gev_Sqrt_TS30_100
gev_TEVCAT_FLAG
gev_Time_Peak
gev_Unc_Cutoff
gev_Unc_Energy_Flux100
gev_Unc_Exp_Index
gev_Unc_Flux1000
gev_Unc_Flux10000_100000
gev_Unc_Flux10

,index,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,s_MAIN_ID,s_class,s_simbad
0,19,b'0FGL J0007.4+7303',b'1FGL J0007.0+7303 ',b'1FHL J0007.3+7303 ',b'2FGL J0007.0+7303 ',b'LAT PSR J0007+7303 ',b' ',b'1AGL J0006+7311',b' ',b'EGR J0008+7308 ',...,b'pl',b'rC5JCj',227,b'TeV J0006+729',1,b'TeV J0006+7259',b'gal',[],[],0
1,33,b' ',b' ',b' ',b' ',b'RBS 0030 ',b' ',b' ',b' ',b' ',...,b'pl',b'70hRXZ',213,b'TeV J0013-188',2,b'TeV J0013-1853',b'egal',[],[],0
2,59,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,b'pl',b'xCyAjF',209,b'TeV J0025+641',3,b'TeV J0025+6410',b'gal',[],[],0
3,75,b'0FGL J0033.6-1921',b'1FGL J0033.5-1921 ',b'1FHL J0033.6-1921 ',b'2FGL J0033.5-1921 ',b'KUV 00311-1938 ',b' ',b' ',b' ',b' ',...,b'none',b'PKJZqs',239,b'TeV J0033-193',4,b'TeV J0033-1921',b'egal',[],[],0
4,78,b'0FGL J0036.7+5951',b'1FGL J0035.9+5951 ',b'1FHL J0035.9+5950 ',b'2FGL J0035.8+5951 ',b'1ES 0033+595 ',b' ',b' ',b' ',b' ',...,b'none',b'eNYfrm',229,b'TeV J0035+598',5,b'TeV J0035+5947',b'egal',[],[],0


In [10]:
gevtev_simbadclasses.shape

(199, 169)

In [16]:
# пересечение xmm без спектров из симбада (только классы)
matrix = create_matrix_common2(gevtev_simbadclasses, data_xmm, _epsilon2, "gev_RAJ2000", "gev_DEJ2000", "xmm_SC_RA", "xmm_SC_DEC")
matrix.shape

(199, 499266)
(199, 1) (499266, 1)


(199, 499266)

In [17]:
!free -h

              total        used        free      shared  buff/cache   available
Память:        3,8G        1,3G        2,2G        135M        348M        2,1G
Подкачка:        3,9G        3,3G        597M


In [18]:
pd_common_gevtevxmm = create_common_data(gevtev_simbadclasses, data_xmm, matrix, "gev_RAJ2000", "gev_DEJ2000", "xmm_SC_RA", "xmm_SC_DEC")

100%|██████████| 1680/1680 [06:38<00:00,  4.21it/s]


In [19]:
print("There are " + str(len(pd_common_gevtevxmm)) + " common objects in the gev-tev catalog with xmm.")

There are 1689 common objects in the gev-tev catalog with xmm.


In [20]:
pd_common_gevtevxmm.to_csv("data/gevtev_simbadclasses_xmm.txt", sep='\t')

In [21]:
pd_common_gevtevxmm = pd.read_csv("data/gevtev_simbadclasses_xmm.txt", sep='\t')
pd_common_gevtevxmm = pd_common_gevtevxmm.loc[:, ~pd_common_gevtevxmm.columns.str.contains('^Unnamed')]
pd_common_gevtevxmm = pd_common_gevtevxmm.loc[:, ~pd_common_gevtevxmm.columns.str.contains('^marked')]
pd_common_gevtevxmm.to_csv('data/gevtevxmm_s.txt',sep = '\t')

In [22]:
print(len(pd_common_gevtevxmm['gev_RAJ2000'].unique()))
print(len(gevtev_simbadclasses['gev_RAJ2000'].unique()))

105
171
